This notebook seeks to explore whether a portfolio's total ROI can be improved by holding a certain amount of cash reserves to 'buy the dip'--attempting to time the market by buying at relative low points rather than holding for a longer duration.

## 0. Import Modules and Data

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import pandas_datareader.data as web
import datetime as dt
import Portfolio as port

## 1. Set-up Portfolio

In [2]:
# Define start date and end date

start_date = '2015-01-01'

end_date = '2020-01-05'

In [3]:
# Define Shares

# Provide the name of the ticker and type (Equity or Bond)

#VOO Vanguard ETF is serving as a proxy for individual stocks/bonds

voo = port.Share('VOO', 'Equity', start_date,end_date)

In [4]:
shares_list = [voo]

shares_dict = {}

for share in shares_list:
    shares_dict[share] = share.type

In [5]:
# Define Strategy

# Provide the equity distribution, the bond distriubtion, cash distribution, and the threshold
strat = port.Strategy(100,0,0,1)

In [6]:
# Lets set up a rolling history of the last 90 days

# Actions measured in days, not minutes...

# the more the share value drops, the more cash we put in --wait until the panic has subsided

In [38]:
portfolio = port.Portfolio(shares_dict, div_reinvest = False)

portfolio.initial_buy(500, strat, start_date)

In [39]:

time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))

# set a counter to avoid buying into shares more than once a week
counter = 0
for day in time_period:
    counter +=1
    portfolio.reinvest_divs(day)
    portfolio.get_asset_values(day)
    for share in portfolio.shares:
        share.get_rolling(day)
        last_week = day - dt.timedelta(days = 7)
        prev_week = last_week - dt.timedelta(days = 7)
        week_avg = share.rolling_90_days[share.rolling_90_days.index>=last_week]['Low'].mean()
        ninety_day_avg = share.rolling_90_days.Low.mean()
        if (week_avg-ninety_day_avg)/ninety_day_avg < -0.05 and counter >=7:
            buy_amount = (((abs((week_avg-ninety_day_avg)/share.rolling_90_days.Low.mean())+1)**2)-1)*portfolio.cash_bal
            portfolio.buy(share, buy_amount, day)
            counter = 0
portfolio_history = portfolio.get_hist_df()
portfolio_history['Date'] = pd.to_datetime(portfolio_history['Date']) # Convert date to datetime object
portfolio_history.set_index('Date', inplace = True)             

In [40]:
portfolio_history

,Total Value,Equities,Bonds
Date,,,
2015-01-01,500.000000,500.000000,0
2015-01-02,500.000000,500.000000,0
2015-01-03,491.215499,491.215499,0
2015-01-04,491.215499,491.215499,0
2015-01-05,491.215499,491.215499,0
...,...,...,...
2020-01-01,865.227262,835.251670,0
2020-01-02,865.227262,835.251670,0
2020-01-03,859.125631,829.150039,0


In [ ]:
# Run without trying to time the market

In [ ]:
portfolio = port.Portfolio(shares_dict, div_reinvest = True)

portfolio.initial_buy(500, strat, start_date)

In [41]:

time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))

# set a counter to avoid buying into shares more than once a week
counter = 0
for day in time_period:
    counter +=1
    portfolio.reinvest_divs(day)
    portfolio.get_asset_values(day)
#    for share in portfolio.shares:
#         share.get_rolling(day)
#         last_week = day - dt.timedelta(days = 7)
#         prev_week = last_week - dt.timedelta(days = 7)
#         week_avg = share.rolling_90_days[share.rolling_90_days.index>=last_week]['Low'].mean()
#         ninety_day_avg = share.rolling_90_days.Low.mean()
#         if (week_avg-ninety_day_avg)/ninety_day_avg < -0.05 and counter >=7:
#             buy_amount = (((abs((week_avg-ninety_day_avg)/share.rolling_90_days.Low.mean())+1)**2)-1)*portfolio.cash_bal
#             portfolio.buy(share, buy_amount, day)
#             counter = 0
portfolio_history = portfolio.get_hist_df()
portfolio_history['Date'] = pd.to_datetime(portfolio_history['Date']) # Convert date to datetime object
portfolio_history.set_index('Date', inplace = True)             

In [42]:
portfolio_history

,Total Value,Equities,Bonds
Date,,,
2015-01-01,500.000000,500.000000,0
2015-01-02,500.000000,500.000000,0
2015-01-03,491.215499,491.215499,0
2015-01-04,491.215499,491.215499,0
2015-01-05,491.215499,491.215499,0
...,...,...,...
2020-01-01,928.888542,835.251670,0
2020-01-02,928.888542,835.251670,0
2020-01-03,922.786911,829.150039,0


In [ ]:
voo.get_data()

In [ ]:
voo.get_rolling(start_date)

In [ ]:
voo.rolling_90_days.Low.mean()

In [ ]:
# Create a loop for different cash allocations

# This is largely the same as above, however, the strat (strategy) will change in each iteration

ini_amount = 500
 
for i in range(0,105,5):
    strat = port.Strategy(i,0,100-i,1) 
    
    portfolio = port.Portfolio(shares_dict)

    portfolio.initial_buy(ini_amount, strat, start_date)
    
    time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))

    for day in time_period:
        portfolio.reinvest_divs(day)
        portfolio.get_asset_values(day)
        if portfolio.asset_split['Equities'] > strat.equity_distribution+strat.threshold:
            sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*((portfolio.asset_split['Equities']-strat.equity_distribution)/100)
            sell_amt_per = sell_amt/len(portfolio.equities)
            for share in portfolio.equities: # sell equities and buy more bonds
                portfolio.sell(share, sell_amt_per, day)
            for share in portfolio.bonds:
                portfolio.buy(share, sell_amt_per, day)

        if portfolio.asset_split['Bonds'] > strat.bond_distribution+strat.threshold:
            sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*(portfolio.asset_split['Bonds']-strat.bond_distribution)
            sell_amt_per = sell_amt/len(portfolio.bonds)
            for share in portfolio.bonds: # sell bonds and buy more equities
                portfolio.sell(share, sell_amt_per, day)
            for share in portfolio.bonds:
                portfolio.buy(share, sell_amt_per, day)
    portfolio_history = portfolio.get_hist_df()
    portfolio_history['Date'] = pd.to_datetime(portfolio_history['Date']) # Convert date to datetime object
    portfolio_history.set_index('Date', inplace = True) 
    
    # Create a new column of the allocation dataframe based on the total value of the most recent portfolio simulation
    allocation_df[str(i)+'% Equities:'+str(100-i)+'% Bonds'] = portfolio_history['Total Value']
    
    # Loop through each year to calculate a ROI in each year 
    
    portfolio_history['Year'] = portfolio_history.index.strftime('%Y')
    
    years = list(portfolio_history['Year'].unique())
    
    for year in years:
        subset = portfolio_history[portfolio_history['Year']==year]
        allocation_roi_df.at[year,str(i)+'% Equities:'+str(100-i)+'% Bonds'] = ((subset['Total Value'].iat[-1]/subset['Total Value'].iat[0])-1)*100
    allocation_roi_df.at['Period Total',str(i)+'% Equities:'+str(100-i)+'% Bonds'] = ((portfolio_history['Total Value'].iat[-1]/portfolio_history['Total Value'].iat[0])-1)*100